### importing and reading

In [55]:
# import modules
import pandas as pd
import seaborn as sns
import numpy as np

# Fixing random state for reproducibility
np.random.seed(387)

In [41]:
# read data
characters = pd.read_csv('datasets/Characters.csv', sep=';')
hp1 = pd.read_csv('datasets/Harry Potter 1.csv', sep=';')
hp2 = pd.read_csv('datasets/Harry Potter 2.csv', sep=';')
hp3 = pd.read_csv('datasets/Harry Potter 3.csv', sep=';')

### getting column names

In [56]:
# column names
print(characters.columns,'\n')
print(hp1.columns)

Index(['Id', 'Name', 'Gender', 'Job', 'House', 'Wand', 'Patronus', 'Species',
       'Blood status', 'Hair colour', 'Eye colour', 'Loyalty', 'Skills',
       'Birth', 'Death'],
      dtype='object') 

Index(['Character', 'Sentence'], dtype='object')


# Plots :
### Characters with the most sentences
#### top 15 of the first 3 movies 

In [57]:
fig, ax = plt.subplots()

ax.barh(Character, Sentence)
ax.set_xlabel('top 15 of the first 3 movies ')
ax.set_title('Characters with the most sentences')

NameError: name 'plt' is not defined